In [7]:
import pandas.io.data as pweb
import datetime
import requests

# Function for comparing long, medium, and short term moving averages 
def price_trend(ticker):
    start_date = datetime.datetime.now() + datetime.timedelta(-45) # Start date will be 45 days ago in order to ensure 30 prices
    end_date = datetime.datetime.now() # Today's date and time
    
    finance=pweb.DataReader(ticker, 'yahoo', start_date, end_date)['Open']  # Get Opening Price about the ticker fron Yahoo Fin.
    #print(finance)  # Can print out the entire dataframe of Opening Price data 

    long_ma=finance[-30:].sum()/len(finance[-30:])   # Add last 30 days and divide for long term moving avg
    print("  Long Term Moving Average:",long_ma)

    med_ma=finance[-20:].sum()/len(finance[-20:])
    print("Medium Term Moving Average:",med_ma)

    short_ma=finance[-10:].sum()/len(finance[-10:])
    print(" Short Term Moving Average:",short_ma)
    
    trend_buy=False  # Boolean variable to indicate wether the moving averages point to a buy (true) or not (false)
    
    if (short_ma>med_ma and short_ma>long_ma):    # Set Buy to True if short term moving average is greater than med and long
        trend_buy=True
        
    return trend_buy     # Return true or false

# Function to scrap HTML of news sites to get amount of news mentions for each company
def stock_frequency(company):
    sources=['http://www.nytimes.com/pages/technology/index.html',    
             'https://www.thestreet.com/tech',
             'http://www.investors.com/technology/']   # Websites to pull news information

    tickers= {"Google":0,"Facebook":0,"Microsoft":0,"Verizon":0,"IBM":0,"Apple":0,"Yahoo":0}

    for source in sources:
        site = requests.get(source)
        site_text=site.text.lower()
        for stock in tickers.keys():
            count=site_text.split(stock.lower())
            tickers[stock]+=len(count)-1
        
    total=0
    for value in tickers.values():
        total+=value
    avg=total/len(tickers)
    print("\nAverage Stock Frequency: ",avg)
    print(company,"Frequency: ", tickers[company],"\n")
    
    frequency_buy=False
    
    if (tickers[company]>avg):
        frequency_buy=True
    
    return frequency_buy

# Function for macd which is 12 day simple moving average - 26 day simple moving average
def macd(ticker):
    startDate = datetime.datetime.now() + datetime.timedelta(-45)
    endDate = datetime.datetime.now()
    finance=pweb.DataReader(ticker,'yahoo', startDate, endDate)['Close']
    twelveDaySma = finance[-12:].sum()/len(finance[-12:])
    twentySixSMA = finance[-26:].sum()/len(finance[-26:])
    macDLine = twelveDaySma - twentySixSMA
    def signalLine():
        counter = 0
        signal = 0
        while (counter < 9):
            numList = list()
            dateStart = datetime.datetime.now() + datetime.timedelta(-45)
            dateEnd = datetime.datetime.now()
            secondFinance=pweb.DataReader(ticker, 'google',  dateStart, dateEnd)['Open']
            twelveDay = finance [(-12-counter):].sum()/len(finance[(-12-counter):])
            twentySixDay = finance [(-26-counter):].sum()/len(finance[(-26-counter):])
            macDNumToAvg = twelveDay - twentySixDay
            numList.append(macDNumToAvg)
            signal = sum(numList)/len(numList)
            counter += 1
            
        macd_buy=False
        
        if (macDLine > signal):
            macd_buy=True
            print ('\nMacD is positive.')
        else:
            print ('\nMacD is not looking good.')
        
        return macd_buy
        
    return signalLine()


def main():
    # Display program information
    print("Stock Buy Decision Application v1.0\n")
    print("This program will help you determine whether you should buy a particular stock based on information from")
    print("Yahoo Finance and various news sites.\n")
    print("Currently, this application only works for: Apple, Facebook, Google, IBM, Microsoft, Verizon, and Yahoo.\n")
    
    company_input=input("Enter the company name: ")   # User input for the name of the company
    ticker_input=input("Enter the company's ticker symbol: ")   # User input for the company's ticker symbol
    
    # Grab boolean values from the functions
    frequency=stock_frequency(company_input)
    trend=price_trend(ticker_input)
    mac=macd(ticker_input)
    
    # Display the final buy decision for the user
    if(frequency==True and trend==True and mac==True):
        print("\nBuy the stock! Financial information indicates that the stock price will likely rise.")
    elif(frequency==True and trend==True and mac == False):
        print ("Moderate buy. Macd is in a downtrend.")
    else:
        print("\nDo not buy the stock! Financial information indicates that the stock price will likely drop.")

main()

Stock Buy Decision Application v1.0

This program will help you determine whether you should buy a particular stock based on information from
Yahoo Finance and various news sites.

Currently, this application only works for: Apple, Facebook, Google, IBM, Microsoft, Verizon, and Yahoo.

Enter the company name: Apple
Enter the company's ticker symbol: AAPL

Average Stock Frequency:  28.714285714285715
Apple Frequency:  62 

  Long Term Moving Average: 110.33799979999998
Medium Term Moving Average: 110.18199965000001
 Short Term Moving Average: 110.71399999999998

MacD is positive.

Buy the stock! Financial information indicates that the stock price will likely rise.
